In [1]:
import random

# Arguments: Reward + Disount should be equal to 1
REWARD = -0.10 # constant reward for non-terminal states
Y = 0.90
MAX_ERROR = 10**(-3)

# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # South, West, North, East Since pickup and drop off can be either if these dirtections
NUM_ROW = 3
NUM_COL = 4
UTILITY = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]
policy = [[random.randint(0, 3) for j in range(NUM_COL)] for i in range(NUM_ROW)] # construct a random policy
SOLUTION = ""

# Visualizing the enviroment.
def printEnviron(arr, policy=False):
    result = ""
    for row in range(NUM_ROW):
        result += "|"
        for column in range(NUM_COL):
            # Position for T cannot be the initial position
            if row == column == 1:
                value = "WALL"

            # Setting the boundaries for the rows and columns to be used as steps 
            elif row <= 1 and column == 5:
                value = "+1" if row == 0 else "-1"
            else:
                value = ["South", "West", "North", "East"][arr[row][column]]
            result += " " + value[:5].ljust(5) + " |" # format
        result += "\n"
    return result

# Performing some simplified value iteration steps to get an approximation of the utilities
def policyEval(policy, UTILITY):
    while True:
        nextUtility = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]
        error = 0

        # Looping through the rows and columns to help evaluate the policy
        for row in range(NUM_ROW):
            for column in range(NUM_COL):
                if (row <= 1 and column == 3) or (row == column == 1):
                    continue
                nextUtility[row][column] = calculateUtility(UTILITY, row, column, policy[row][column]) # simplified Bellman update
                error = max(error, abs(nextUtility[row][column]-UTILITY[row][column]))
        UTILITY = nextUtility
        if error < MAX_ERROR * (1-Y) / Y:
            break
    return UTILITY    

# Getting the utility of the state reached by performing the given action from the given state
def getUtility(UTILITY, row, column, action):
    dr, dc = ACTIONS[action]
    newR, newC = row+dr, column+dc
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR == newC == 1): # collide with the boundary or the wall
        return UTILITY[row][column]
    else:
        return UTILITY[newR][newC]

# Calculating the utility of a state given the action
def calculateUtility(UTILITY, row, column, action):
    u = REWARD
    u += 0.1 * Y * getUtility(UTILITY, row, column, (action-1)%4)
    u += 0.8 * Y * getUtility(UTILITY, row, column, action)
    u += 0.1 * Y * getUtility(UTILITY, row, column, (action+1)%4)
    return u

def policyIter(policy, UTILITY, tax, p1p, p2p):
    steps = 0;
    while True:
        UTILITY = policyEval(policy, UTILITY)
        unchanged = True
        
        changes = random.randint(0, 1)

        path = "tax: " , tax ," => "
        # Iterating through the policy
        for row in range(NUM_ROW):
            for column in range(NUM_COL):
                
                
                if (row <= 1 and column == 3) or (row == column == 1):
                    continue
                maxAction, maxU = None, -float("inf")

                for action in range(NUM_ACTIONS):
                    
                    u = calculateUtility(UTILITY, row, column, action)

                    if u > maxU:
                        maxAction, maxU = action, u
                        
                if maxU > calculateUtility(UTILITY, row, column, policy[row][column]):
                    policy[row][column] = maxAction # the action that maximizes the utility
                    unchanged = False
                    steps = steps +1
        if unchanged:
            break

    p1d = [random.randint(1, 5),random.randint(1, 5)]
    p2d = [random.randint(1, 5),random.randint(1, 5)]
    if changes == 0:
        changes = random.randint(0, 1)
        path = ''.join(map(str, path)) , "p2p: ",p2p," => "
        
        if changes == 0:
            path = ''.join(map(str, path)) + "p1p: ",p1p," => "
            path = ''.join(map(str, path)) , "p2d: ",p2d," => "
            path = ''.join(map(str, path)) , "p1d: ",p1d
        elif changes == 1:
            path = ''.join(map(str, path)) , "p2d: ",p2d," => "
            path = ''.join(map(str, path)) , "p1p: ",p1p," => "
            path = ''.join(map(str, path)) , "p1d: ",p1d
    else:
        path = ''.join(map(str, path)) , "p1p: ",p1p," => "
        changes = random.randint(0, 1)

        if changes == 0:
            path = ''.join(map(str, path)) , "p2p: ",p2p," => "
            path = ''.join(map(str, path)) , "p1d: ",p1d," => "
            path = ''.join(map(str, path)) , "p2d: ",p2d
        elif changes == 1:
            path = ''.join(map(str, path)) , "p1d: ",p1d," => "
            path = ''.join(map(str, path)) , "p2p: ",p2p," => "
            path = ''.join(map(str, path)) , "p2d: ",p2d

    printEnviron(policy)
    print("Steps = ",steps)
    listToStr = ' '.join([str(elem) for elem in path])
    print("The path is: ", path)
    print("The optimal path is:")
    return policy

if __name__ == "__main__":
    
    tax = [2,4]
    p1p = [1,5]
    p2p = [5,1]
    print("Episode 1 from figure 1:")
    print("The tax location is ", tax)
    print("The first passenger location is ", p1p)
    print("The second pasenger location is ", p2p)
    
    # Getting the original policy from figure one.
    policy = [[3, 1, 2, 0], [1, 1, 2, 3], [0, 3, 0, 3]]

    # Printing the optimal path
    policy = policyIter(policy, UTILITY, tax, p1p, p2p)
    SOLUTION = printEnviron(policy)
    print(SOLUTION)

    # Iterating throuh the policy.
    for i in range(4):
        tax = [random.randint(1, 5),random.randint(1, 5)]
        p1p = [random.randint(1, 5),random.randint(1, 5)]
        p2p = [random.randint(1, 5),random.randint(1, 5)]

        print("Episode ", i+2,":")
        print("The tax location is ", tax)
        print("The first passenger location is ", p1p)
        print("The second pasenger location is ", p2p)
        policy = [[random.randint(0, 3) for j in range(NUM_COL)] for i in range(NUM_ROW)]
        # Printing the optimal policy

        policy = policyIter(policy, UTILITY, tax, p1p, p2p)
        SOLUTION = printEnviron(policy)
        print(SOLUTION)

Episode 1 from figure 1:
The tax location is  [2, 4]
The first passenger location is  [1, 5]
The second pasenger location is  [5, 1]
Steps =  10
The path is:  ('tax: [2, 4] => p2p: [5, 1] => p2d: [3, 2] => p1p: [1, 5] => ', 'p1d: ', [4, 4])
The optimal path is:
| East  | East  | East  | South |
| North | WALL  | North | East  |
| North | East  | North | West  |

Episode  2 :
The tax location is  [1, 1]
The first passenger location is  [4, 1]
The second pasenger location is  [3, 3]
Steps =  6
The path is:  ('tax: [1, 1] => p1p: [4, 1] => p2p: [3, 3] => p1d: [1, 1] => ', 'p2d: ', [5, 2])
The optimal path is:
| East  | East  | East  | West  |
| North | WALL  | North | East  |
| North | East  | North | West  |

Episode  3 :
The tax location is  [2, 3]
The first passenger location is  [1, 2]
The second pasenger location is  [3, 1]
Steps =  11
The path is:  ('tax: [2, 3] => p1p: [1, 2] => p1d: [2, 1] => p2p: [3, 1] => ', 'p2d: ', [5, 4])
The optimal path is:
| East  | East  | East  | West  |